<a href="https://colab.research.google.com/github/eastmountaincode/DSC/blob/main/ohioPopulationListTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is documentation of the process used to get the list of cities at this link: https://www.ohio-demographics.com/cities_by_population into a useable format for the Emergency Room/Urgent Care review data project.

## Initialize

I copied the data on the ohio-demographics.com website into a spreadsheet and bounced it to a CSV.

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ERURproject/ohiobypop.csv")

Mounted at /content/drive


## Problems

The data has several problems (that we can fix). 

1. Sometimes, a single column contains two cities. The rank column contains the word "TIE", and city column contains two cities separated by the word "and", and the population is the same for both cities.

In [2]:
df.loc[125]

Rank                           126 TIE 
City          Sharonville and Willowick
Population                       14,133
Name: 125, dtype: object

2. Sometimes, a single column contains THREE OR FOUR cities!

In [3]:
df.loc[878]

Rank                                      879 TIE 
City          Pitsburg, New Marshfield, and Bascom
Population                                     330
Name: 878, dtype: object

3. Sometimes, a city had two names, one of them being in parentheses.

In [4]:
df.loc[883]

Rank                        884
City          Hartford (Croton)
Population                  323
Name: 883, dtype: object

## The fix

In [5]:
tempdf = pd.DataFrame()

for i in range(len(df)):
  #For these cases where there are 2, 3, or 4 cities in one row
  if "TIE" in df.loc[i, "Rank"]:
    manyCities = df.loc[i, "City"]
    
    #IF THERE ARE THREE OR FOUR CITIES IN A TIE
    #If there are more than two cities in a row, the city column will contain a comma
    if "," in manyCities:
      #Split the city names by commas
      allThreeorFour = manyCities.split(", ")
      #On the last city name, remove the word "and"
      allThreeorFour[-1] = allThreeorFour[-1].strip('and ')
      
      #SPECIAL ANNOYING CASE
      #Here we have a problem that had to be handled manually
      #Whenever I came across a city with multiple names, I googled both names and determined which was more meaningful by looking at the results
      if 'Gann' in allThreeorFour[-1]:
        allThreeorFour[-1] = allThreeorFour[-1].strip('(Brinkhaven)')
        allThreeorFour[-1] = allThreeorFour[-1].strip(' ')

      for x in allThreeorFour:
        rankString = df.loc[i, "Rank"]
        rankString = rankString.replace("TIE", "")
        rankString = rankString.replace(" ", "")

        lostCity = x

        cityPopulation = df.loc[i, "Population"]

        newRow = {'Rank': rankString, 'City': lostCity, 'Population': cityPopulation}
        tempdf = tempdf.append(newRow, ignore_index = True)
         
    #IF THERE ARE ONLY TWO CITIES IN A TIE
    else:
      twoCities = manyCities.split(' and ')

      # print(twoCities)
      
      #SPECIAL ANNOYING CASE
      #Dual name problem handled manually
      if 'Rome (Stout)' in twoCities:
        twoCities[0] = twoCities[0].strip('Rome (')
        twoCities[0] = twoCities[0].strip(')')
      
      for x in twoCities:
        rankString = df.loc[i, "Rank"]
        rankString = rankString.replace("TIE", "")
        rankString = rankString.replace(" ", "")

        lostCity = x

        cityPopulation = df.loc[i, "Population"]

        newRow = {'Rank': rankString, 'City': lostCity, 'Population': cityPopulation}
        tempdf = tempdf.append(newRow, ignore_index = True)

  #IF ONLY ONE CITY
  else:
    #SPECIAL ANNOYING CASE
    #Dual name problem handled manually
    if df.loc[i, "City"] == "Hartford (Croton)":
      rankString = df.loc[i, "Rank"]
      cityName = "Hartford"
      cityPopulation = df.loc[i, "Population"]
      newRow = {'Rank': rankString, 'City': cityName, 'Population': cityPopulation}
      tempdf = tempdf.append(newRow, ignore_index = True)

    #Dual name problem handled manually
    elif df.loc[i, "City"] == "Centerville (Thurman)":
      rankString = df.loc[i, "Rank"]
      cityName = "Hartford"
      cityPopulation = df.loc[i, "Population"]
      newRow = {'Rank': rankString, 'City': cityName, 'Population': cityPopulation}
      tempdf = tempdf.append(newRow, ignore_index = True)

    else:
      rankString = df.loc[i, "Rank"]
      cityName = df.loc[i, "City"]
      cityPopulation = df.loc[i, "Population"]
      newRow = {'Rank': rankString, 'City': cityName, 'Population': cityPopulation}
      tempdf = tempdf.append(newRow, ignore_index = True)
        

And that's it! Now we have a dataframe that can be iterated over without any problem.

In [6]:
tempdf

,City,Population,Rank
0,Columbus,"878,553",1
1,Cleveland,"385,282",2
2,Cincinnati,"301,394",3
3,Toledo,"276,614",4
4,Akron,"198,051",5
...,...,...,...
1209,Otway,32,"1,032"
1210,Kilbourne,19,"1,033"
1211,Holiday City,17,"1,034"
1212,Beulah Beach,11,"1,035"


In [ ]:
from google.colab import files
tempdf.to_csv('ohioPopTutorial.csv')
files.download('ohioPopTutorial.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>